# スタッキング001

In [1]:
import torch
# GPUの名称、CUDA Compute Capabilityを取得
print(torch.cuda.get_device_name())

NVIDIA GeForce RTX 2080 Ti


In [2]:
import cupy as cp
import numpy as np
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import scipy.stats as st
import pickle
import optuna
import re
import gc

from lightgbm import LGBMClassifier

from sklearn import metrics
from sklearn.tree import plot_tree
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.impute import SimpleImputer
from sklearn_pandas import DataFrameMapper
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss
from sklearn.metrics import mean_squared_log_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.metrics import roc_auc_score
from sklearn.datasets import make_moons
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import mutual_info_classif
from sklearn.experimental import enable_iterative_imputer

from keras.layers import Dense, Dropout
from keras.models import Sequential

from six import StringIO

from IPython.display import Image
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
import math
from scipy import stats
import numpy as np
import pandas as pd
import tensorflow as tf

from datetime import datetime
from tensorflow import feature_column
from tensorflow.keras import layers

from tqdm import tqdm

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

# File system manangement
import os

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

import xgboost as xgb
from sklearn.metrics import log_loss
from xgboost.sklearn import XGBClassifier

from Modules import Modules
from Baseline import Baseline
from FeatureEngineering2 import FeatureEngineering2
from FeatureEngineering import FeatureEngineering
from models import Model1Lgb, Model1Xgb, Model1NN, Model2Linear

%matplotlib inline
gc.enable()

print("OK\n")

OK



In [3]:
path = './home-credit-default-risk/'

## 共通関数定義

In [4]:
### LightGBM
params_lgb = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'nthread': -1
    ,'n_estimators': 5000
    ,'learning_rate': 0.01
    ,'max_depth': 11
    ,'num_leaves': 58
    ,'colsample_bytree': 0.613
    ,'subsample': 0.708
    ,'max_bin': 407
    ,'reg_alpha': 3.564
    ,'reg_lambda': 4.930
    ,'min_child_weight': 6
    ,'min_child_samples': 165
    ,'silent': -1
    ,'verbose': -1
    ,'importance_type': 'gain'
}
### Xgboost
params_xgb = {
        'booster': 'gbtree'
        ,'objective': 'binary:logistic'
        ,'silent': 1
        ,'random_state': 17
        ,'eval_metric': 'auc'
        ,'eta': 0.1
        ,'max_depth': 8
        ,'subsample': 0.9
        ,'colsample_state': 71
        ,'colsample_bytree': 1.0
        ,'colsample_bylevel': 0.3
        ,'gamma': 0
        ,'lambda': 1
        ,'alpha': 0
        ,'min_child_weight': 1
        ,'gpu_id': 0
        ,'tree_method': 'gpu_hist'
}

In [5]:
# 学習データに対する「目的変数を知らない」予測値と、テストデータに対する予測値を返す関数
def predict_cv(model, train_x, train_y, test_x):
    preds = []
    preds_test = []
    va_idxes = []

    kf = KFold(n_splits=4, shuffle=True, random_state=71)

    # クロスバリデーションで学習・予測を行い、予測値とインデックスを保存する
    for i, (tr_idx, va_idx) in enumerate(kf.split(train_x)):
        tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
        tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
        model.fit(tr_x, tr_y, va_x, va_y)
        pred = model.predict(va_x)
        print(pred)
        preds.append(pred)
        pred_test = model.predict(test_x)
        preds_test.append(pred_test)
        va_idxes.append(va_idx)

    # バリデーションデータに対する予測値を連結し、その後元の順序に並べ直す
    va_idxes = np.concatenate(va_idxes)
    preds = np.concatenate(preds, axis=0)
    order = np.argsort(va_idxes)
    pred_train = preds[order]

    # テストデータに対する予測値の平均をとる
    preds_test = np.mean(preds_test, axis=0)

    return pred_train, preds_test

In [6]:
params_base = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'nthread': -1
    ,'n_estimators': 5000
    ,'learning_rate': 0.01
    ,'max_depth': 11
    ,'colsample_bytree': 0.613
    ,'subsample': 0.708
    ,'max_bin': 407
    ,'reg_alpha': 3.564
    ,'reg_lambda': 4.930
    ,'min_child_samples': 165
    ,'silent': -1
    ,'verbose': -1
    ,'importance_type': 'gain'
}



def objective(trial):
    # 探索するパラメータ
    param_tuning = {
            "num_leaves": trial.suggest_int("num_leaves", 8, 256)
            ,"min_child_samples": trial.suggest_int("min_child_samples", 5, 200)
            ,"min_sum_hessian_in_leaf": trial.suggest_float("min_sum_hessian_in_leaf", 1e-5, 1e-2, log=True)
            ,"feature_fraction": trial.suggest_float("feature_fraction", 0.5, 1.0)
            ,"bagging_fraction": trial.suggest_float("bagging_fraction", 0.5, 1.0)
            ,"lambda_l1": trial.suggest_float("lambda_l1", 1e-2, 1e+2, log=True)
            ,"lambda_l2": trial.suggest_float("lambda_l2", 1e-2, 1e+2, log=True)
    }
    param_tuning.update(params_base)

    # モデル学習・評価:
    list_metrics = []
    cv = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=123).split(x_train, y_train))
    list_fold = [0]
    for nfold in list_fold:
        idx_tr, idx_va = cv[nfold][0], cv[nfold][1]
        x_tr, y_tr = x_train.loc[idx_tr, :], y_train[idx_tr]
        x_va, y_va = x_train.loc[idx_va, :], y_train[idx_va]
        model = lgb.LGBMClassifier(**params)
        model.fit(x_tr
                    ,y_tr
                    ,eval_set=[(x_tr, y_tr), (x_va, y_va)]
                    ,early_stopping_rounds=100
                    ,verbose=0
        )
        y_va_pred = model.predict_proba(x_va)[:,1]
        metric_va = roc_auc_score(y_va, y_va_pred)
        list_metrics.append(metric_va)

    # 評価指数の算出
    metrics = np.mean(list_metrics)

    return metrics

## インスタンス生成

In [7]:
baseline = Baseline('SK_ID_CURR')
modules = Modules('SK_ID_CURR')
FeatureEngineering = FeatureEngineering('SK_ID_CURR')
FeatureEngineering2 = FeatureEngineering2('SK_ID_CURR')

## 特徴量生成1

In [8]:
### bureau_balance
bureau = FeatureEngineering.bureau_and_balance(path)
bureau.head()

Memory usage of dataframe: 4114.69 MB
Memory usage after optimization: 1189.22 MB
Decreased by 71.1%


,BURO_SK_ID_CURR_MIN,BURO_SK_ID_CURR_MAX,BURO_SK_ID_CURR_SIZE,BURO_SK_ID_CURR_MEAN,BURO_SK_ID_CURR_VAR,BURO_SK_ID_CURR_SUM,BURO_DAYS_CREDIT_MIN,BURO_DAYS_CREDIT_MAX,BURO_DAYS_CREDIT_SIZE,BURO_DAYS_CREDIT_MEAN,...,CLOSED_Last_status_3_MEAN_SIZE,CLOSED_Last_status_3_MEAN_MEAN,CLOSED_Last_status_3_MEAN_VAR,CLOSED_Last_status_3_MEAN_SUM,CLOSED_Last_status_4_MEAN_MIN,CLOSED_Last_status_4_MEAN_MAX,CLOSED_Last_status_4_MEAN_SIZE,CLOSED_Last_status_4_MEAN_MEAN,CLOSED_Last_status_4_MEAN_VAR,CLOSED_Last_status_4_MEAN_SUM
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,100001,100001,7,100001.0,0.0,700007,-1572,-49,7,-735.000,...,4.0,0.000000,0.000000,0.0,0.000000,0.000000,4.0,0.000000,0.000000,0.0
100002,100002,100002,8,100002.0,0.0,800016,-1437,-103,8,-874.000,...,6.0,0.000000,0.000000,0.0,0.000000,0.000000,6.0,0.000000,0.000000,0.0
100003,100003,100003,4,100003.0,0.0,400012,-2586,-606,4,-1401.000,...,3.0,1.400391,0.700195,0.0,1.400391,1.400391,3.0,1.400391,0.700195,0.0
100004,100004,100004,2,100004.0,0.0,200008,-1326,-408,2,-867.000,...,2.0,1.400391,0.700195,0.0,1.400391,1.400391,2.0,1.400391,0.700195,0.0
100005,100005,100005,3,100005.0,0.0,300015,-373,-62,3,-190.625,...,1.0,0.000000,0.700195,0.0,0.000000,0.000000,1.0,0.000000,0.700195,0.0


In [9]:
bureau = bureau.replace([np.inf, -np.inf], np.nan)
###bureau = bureau.fillna(bureau.max() + (bureau.max() * 0.4))
bureau.head()

,BURO_SK_ID_CURR_MIN,BURO_SK_ID_CURR_MAX,BURO_SK_ID_CURR_SIZE,BURO_SK_ID_CURR_MEAN,BURO_SK_ID_CURR_VAR,BURO_SK_ID_CURR_SUM,BURO_DAYS_CREDIT_MIN,BURO_DAYS_CREDIT_MAX,BURO_DAYS_CREDIT_SIZE,BURO_DAYS_CREDIT_MEAN,...,CLOSED_Last_status_3_MEAN_SIZE,CLOSED_Last_status_3_MEAN_MEAN,CLOSED_Last_status_3_MEAN_VAR,CLOSED_Last_status_3_MEAN_SUM,CLOSED_Last_status_4_MEAN_MIN,CLOSED_Last_status_4_MEAN_MAX,CLOSED_Last_status_4_MEAN_SIZE,CLOSED_Last_status_4_MEAN_MEAN,CLOSED_Last_status_4_MEAN_VAR,CLOSED_Last_status_4_MEAN_SUM
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,100001,100001,7,100001.0,0.0,700007,-1572,-49,7,-735.000,...,4.0,0.000000,0.000000,0.0,0.000000,0.000000,4.0,0.000000,0.000000,0.0
100002,100002,100002,8,100002.0,0.0,800016,-1437,-103,8,-874.000,...,6.0,0.000000,0.000000,0.0,0.000000,0.000000,6.0,0.000000,0.000000,0.0
100003,100003,100003,4,100003.0,0.0,400012,-2586,-606,4,-1401.000,...,3.0,1.400391,0.700195,0.0,1.400391,1.400391,3.0,1.400391,0.700195,0.0
100004,100004,100004,2,100004.0,0.0,200008,-1326,-408,2,-867.000,...,2.0,1.400391,0.700195,0.0,1.400391,1.400391,2.0,1.400391,0.700195,0.0
100005,100005,100005,3,100005.0,0.0,300015,-373,-62,3,-190.625,...,1.0,0.000000,0.700195,0.0,0.000000,0.000000,1.0,0.000000,0.700195,0.0


In [10]:
### previos_application
previous = FeatureEngineering.previous_application(path)
previous.head()

Memory usage of dataframe: 2434.34 MB
Memory usage after optimization: 818.55 MB
Decreased by 66.4%


,PREV_SK_ID_PREV_MIN,PREV_SK_ID_PREV_MAX,PREV_SK_ID_PREV_SIZE,PREV_SK_ID_PREV_MEAN,PREV_SK_ID_PREV_VAR,PREV_SK_ID_PREV_SUM,PREV_SK_ID_CURR_MIN,PREV_SK_ID_CURR_MAX,PREV_SK_ID_CURR_SIZE,PREV_SK_ID_CURR_MEAN,...,REFUSED_PRODUCT_COMBINATION_POS_other_with_interest_MEAN,REFUSED_PRODUCT_COMBINATION_Card_X_Sell_MEAN,REFUSED_PRODUCT_COMBINATION_POS_mobile_without_interest_MEAN,REFUSED_PRODUCT_COMBINATION_Card_Street_MEAN,REFUSED_PRODUCT_COMBINATION_POS_industry_with_interest_MEAN,REFUSED_PRODUCT_COMBINATION_Cash_Street__low_MEAN,REFUSED_PRODUCT_COMBINATION_POS_industry_without_interest_MEAN,REFUSED_PRODUCT_COMBINATION_Cash_Street__middle_MEAN,REFUSED_PRODUCT_COMBINATION_POS_others_without_interest_MEAN,REFUSED_PRODUCT_COMBINATION_nan_MEAN
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,1369693,1369693,1,1369693.00,2.367471e+12,1369693,100001,100001,1,100001.0,...,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391
100002,1038818,1038818,1,1038818.00,2.367471e+12,1038818,100002,100002,1,100002.0,...,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391
100003,1810518,2636178,3,2281150.25,1.804519e+11,6843451,100003,100003,3,100003.0,...,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391
100004,1564014,1564014,1,1564014.00,2.367471e+12,1564014,100004,100004,1,100004.0,...,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391
100005,1857999,2495675,2,2176837.00,2.033153e+11,4353674,100005,100005,2,100005.0,...,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391


In [11]:
previous = previous.replace([np.inf, -np.inf], np.nan)
###previous = previous.fillna(previous.max() + (previous.max() * 0.4))
previous.head()

,PREV_SK_ID_PREV_MIN,PREV_SK_ID_PREV_MAX,PREV_SK_ID_PREV_SIZE,PREV_SK_ID_PREV_MEAN,PREV_SK_ID_PREV_VAR,PREV_SK_ID_PREV_SUM,PREV_SK_ID_CURR_MIN,PREV_SK_ID_CURR_MAX,PREV_SK_ID_CURR_SIZE,PREV_SK_ID_CURR_MEAN,...,REFUSED_PRODUCT_COMBINATION_POS_other_with_interest_MEAN,REFUSED_PRODUCT_COMBINATION_Card_X_Sell_MEAN,REFUSED_PRODUCT_COMBINATION_POS_mobile_without_interest_MEAN,REFUSED_PRODUCT_COMBINATION_Card_Street_MEAN,REFUSED_PRODUCT_COMBINATION_POS_industry_with_interest_MEAN,REFUSED_PRODUCT_COMBINATION_Cash_Street__low_MEAN,REFUSED_PRODUCT_COMBINATION_POS_industry_without_interest_MEAN,REFUSED_PRODUCT_COMBINATION_Cash_Street__middle_MEAN,REFUSED_PRODUCT_COMBINATION_POS_others_without_interest_MEAN,REFUSED_PRODUCT_COMBINATION_nan_MEAN
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,1369693,1369693,1,1369693.00,2.367471e+12,1369693,100001,100001,1,100001.0,...,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391
100002,1038818,1038818,1,1038818.00,2.367471e+12,1038818,100002,100002,1,100002.0,...,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391
100003,1810518,2636178,3,2281150.25,1.804519e+11,6843451,100003,100003,3,100003.0,...,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391
100004,1564014,1564014,1,1564014.00,2.367471e+12,1564014,100004,100004,1,100004.0,...,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391
100005,1857999,2495675,2,2176837.00,2.033153e+11,4353674,100005,100005,2,100005.0,...,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391


In [12]:
### pos_cash
pos = FeatureEngineering.pos_cash(path)
pos.head()

Memory usage of dataframe: 147.95 MB
Memory usage after optimization: 46.96 MB
Decreased by 68.3%


,POS_SK_ID_PREV_MIN,POS_SK_ID_PREV_MAX,POS_SK_ID_PREV_SIZE,POS_SK_ID_PREV_MEAN,POS_SK_ID_PREV_VAR,POS_SK_ID_PREV_SUM,POS_SK_ID_CURR_MIN,POS_SK_ID_CURR_MAX,POS_SK_ID_CURR_SIZE,POS_SK_ID_CURR_MEAN,...,POS_NAME_CONTRACT_STATUS_Active_MEAN,POS_NAME_CONTRACT_STATUS_Completed_MEAN,POS_NAME_CONTRACT_STATUS_Signed_MEAN,POS_NAME_CONTRACT_STATUS_Approved_MEAN,POS_NAME_CONTRACT_STATUS_Returned_to_the_store_MEAN,POS_NAME_CONTRACT_STATUS_Demand_MEAN,POS_NAME_CONTRACT_STATUS_Canceled_MEAN,POS_NAME_CONTRACT_STATUS_XNA_MEAN,POS_NAME_CONTRACT_STATUS_Amortized_debt_MEAN,POS_COUNT
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,1369693,1851984,9,1584044.50,6.461239e+10,14256401,100001,100001,9,100001.0,...,0.777832,0.222168,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,9
100002,1038818,1038818,19,1038818.00,0.000000e+00,19737542,100002,100002,19,100002.0,...,1.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,19
100003,1810518,2636178,28,2297665.25,1.086316e+11,64334628,100003,100003,28,100003.0,...,0.928711,0.071411,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,28
100004,1564014,1564014,4,1564014.00,0.000000e+00,6256056,100004,100004,4,100004.0,...,0.750000,0.250000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,4
100005,2495675,2495675,11,2495675.00,0.000000e+00,27452425,100005,100005,11,100005.0,...,0.818359,0.090881,0.090881,0.0,0.0,0.0,0.0,0.0,0.0,11


In [13]:
pos = pos.replace([np.inf, -np.inf], np.nan)
###pos = pos.fillna(pos.max() + (pos.max() * 0.4))
pos.head()

,POS_SK_ID_PREV_MIN,POS_SK_ID_PREV_MAX,POS_SK_ID_PREV_SIZE,POS_SK_ID_PREV_MEAN,POS_SK_ID_PREV_VAR,POS_SK_ID_PREV_SUM,POS_SK_ID_CURR_MIN,POS_SK_ID_CURR_MAX,POS_SK_ID_CURR_SIZE,POS_SK_ID_CURR_MEAN,...,POS_NAME_CONTRACT_STATUS_Active_MEAN,POS_NAME_CONTRACT_STATUS_Completed_MEAN,POS_NAME_CONTRACT_STATUS_Signed_MEAN,POS_NAME_CONTRACT_STATUS_Approved_MEAN,POS_NAME_CONTRACT_STATUS_Returned_to_the_store_MEAN,POS_NAME_CONTRACT_STATUS_Demand_MEAN,POS_NAME_CONTRACT_STATUS_Canceled_MEAN,POS_NAME_CONTRACT_STATUS_XNA_MEAN,POS_NAME_CONTRACT_STATUS_Amortized_debt_MEAN,POS_COUNT
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,1369693,1851984,9,1584044.50,6.461239e+10,14256401,100001,100001,9,100001.0,...,0.777832,0.222168,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,9
100002,1038818,1038818,19,1038818.00,0.000000e+00,19737542,100002,100002,19,100002.0,...,1.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,19
100003,1810518,2636178,28,2297665.25,1.086316e+11,64334628,100003,100003,28,100003.0,...,0.928711,0.071411,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,28
100004,1564014,1564014,4,1564014.00,0.000000e+00,6256056,100004,100004,4,100004.0,...,0.750000,0.250000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,4
100005,2495675,2495675,11,2495675.00,0.000000e+00,27452425,100005,100005,11,100005.0,...,0.818359,0.090881,0.090881,0.0,0.0,0.0,0.0,0.0,0.0,11


In [14]:
### instllment_payment
installments = FeatureEngineering.installments_payments(path)
installments.head()

Memory usage of dataframe: 199.49 MB
Memory usage after optimization: 73.84 MB
Decreased by 63.0%


,INS_SK_ID_PREV_MIN,INS_SK_ID_PREV_MAX,INS_SK_ID_PREV_SIZE,INS_SK_ID_PREV_MEAN,INS_SK_ID_PREV_VAR,INS_SK_ID_PREV_SUM,INS_SK_ID_CURR_MIN,INS_SK_ID_CURR_MAX,INS_SK_ID_CURR_SIZE,INS_SK_ID_CURR_MEAN,...,INS_ins_AMT_INSTALMENT_AMT_PAYMENT_MEAN,INS_ins_AMT_INSTALMENT_AMT_PAYMENT_VAR,INS_ins_AMT_INSTALMENT_AMT_PAYMENT_SUM,INS_ins_AMT_PAYMENT_div_AMT_INSTALMENT_MIN,INS_ins_AMT_PAYMENT_div_AMT_INSTALMENT_MAX,INS_ins_AMT_PAYMENT_div_AMT_INSTALMENT_SIZE,INS_ins_AMT_PAYMENT_div_AMT_INSTALMENT_MEAN,INS_ins_AMT_PAYMENT_div_AMT_INSTALMENT_VAR,INS_ins_AMT_PAYMENT_div_AMT_INSTALMENT_SUM,INSTAL_COUNT
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,1369693,1851984,7,1576389.125,6.645846e+10,11034724,100001,100001,7,100001.0,...,0.0,0.0,0.0,1.0,1.0,7,1.0,0.0,7.0,7
100002,1038818,1038818,19,1038818.000,0.000000e+00,19737542,100002,100002,19,100002.0,...,0.0,0.0,0.0,1.0,1.0,19,1.0,0.0,19.0,19
100003,1810518,2636178,25,2290070.250,1.027131e+11,57251754,100003,100003,25,100003.0,...,0.0,0.0,0.0,1.0,1.0,25,1.0,0.0,25.0,25
100004,1564014,1564014,3,1564014.000,0.000000e+00,4692042,100004,100004,3,100004.0,...,0.0,0.0,0.0,1.0,1.0,3,1.0,0.0,3.0,3
100005,2495675,2495675,9,2495675.000,0.000000e+00,22461075,100005,100005,9,100005.0,...,0.0,0.0,0.0,1.0,1.0,9,1.0,0.0,9.0,9


In [15]:
installments = installments.replace([np.inf, -np.inf], np.nan)
###installments = installments.fillna(installments.max() + (installments.max() * 0.4))
installments.head()

,INS_SK_ID_PREV_MIN,INS_SK_ID_PREV_MAX,INS_SK_ID_PREV_SIZE,INS_SK_ID_PREV_MEAN,INS_SK_ID_PREV_VAR,INS_SK_ID_PREV_SUM,INS_SK_ID_CURR_MIN,INS_SK_ID_CURR_MAX,INS_SK_ID_CURR_SIZE,INS_SK_ID_CURR_MEAN,...,INS_ins_AMT_INSTALMENT_AMT_PAYMENT_MEAN,INS_ins_AMT_INSTALMENT_AMT_PAYMENT_VAR,INS_ins_AMT_INSTALMENT_AMT_PAYMENT_SUM,INS_ins_AMT_PAYMENT_div_AMT_INSTALMENT_MIN,INS_ins_AMT_PAYMENT_div_AMT_INSTALMENT_MAX,INS_ins_AMT_PAYMENT_div_AMT_INSTALMENT_SIZE,INS_ins_AMT_PAYMENT_div_AMT_INSTALMENT_MEAN,INS_ins_AMT_PAYMENT_div_AMT_INSTALMENT_VAR,INS_ins_AMT_PAYMENT_div_AMT_INSTALMENT_SUM,INSTAL_COUNT
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,1369693,1851984,7,1576389.125,6.645846e+10,11034724,100001,100001,7,100001.0,...,0.0,0.0,0.0,1.0,1.0,7,1.0,0.0,7.0,7
100002,1038818,1038818,19,1038818.000,0.000000e+00,19737542,100002,100002,19,100002.0,...,0.0,0.0,0.0,1.0,1.0,19,1.0,0.0,19.0,19
100003,1810518,2636178,25,2290070.250,1.027131e+11,57251754,100003,100003,25,100003.0,...,0.0,0.0,0.0,1.0,1.0,25,1.0,0.0,25.0,25
100004,1564014,1564014,3,1564014.000,0.000000e+00,4692042,100004,100004,3,100004.0,...,0.0,0.0,0.0,1.0,1.0,3,1.0,0.0,3.0,3
100005,2495675,2495675,9,2495675.000,0.000000e+00,22461075,100005,100005,9,100005.0,...,0.0,0.0,0.0,1.0,1.0,9,1.0,0.0,9.0,9


In [16]:
### credit_card_balance
credit = FeatureEngineering.credit_card_balance(path)
credit.head()

Memory usage of dataframe: 173.03 MB
Memory usage after optimization: 66.37 MB
Decreased by 61.6%


,CARD_SK_ID_PREV_MIN,CARD_SK_ID_PREV_MAX,CARD_SK_ID_PREV_SIZE,CARD_SK_ID_PREV_MEAN,CARD_SK_ID_PREV_VAR,CARD_SK_ID_PREV_SUM,CARD_SK_ID_CURR_MIN,CARD_SK_ID_CURR_MAX,CARD_SK_ID_CURR_SIZE,CARD_SK_ID_CURR_MEAN,...,CARD_card_AMT_DRAWINGS_CURRENT__AMT_DRAWINGS_POS_CURRENT_VAR,CARD_card_AMT_DRAWINGS_CURRENT__AMT_DRAWINGS_POS_CURRENT_SUM,CARD_NAME_CONTRACT_STATUS_Active_MEAN,CARD_NAME_CONTRACT_STATUS_Completed_MEAN,CARD_NAME_CONTRACT_STATUS_Demand_MEAN,CARD_NAME_CONTRACT_STATUS_Signed_MEAN,CARD_NAME_CONTRACT_STATUS_Sent_proposal_MEAN,CARD_NAME_CONTRACT_STATUS_Refused_MEAN,CARD_NAME_CONTRACT_STATUS_Approved_MEAN,CARD_COUNT
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100006,1489396,1489396,6,1489396.0,0.0,8936376,100006,100006,6,100006.0,...,8.505000e+11,0.0,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,6
100011,1843384,1843384,74,1843384.0,0.0,136410416,100011,100011,74,100011.0,...,4.378378e+08,180000.0,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,74
100013,2038692,2038692,96,2038692.0,0.0,195714432,100013,100013,96,100013.0,...,8.249688e+08,571500.0,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,96
100021,2594025,2594025,17,2594025.0,0.0,44098425,100021,100021,17,100021.0,...,8.505000e+11,0.0,0.411865,0.588379,0.0,0.0,0.0,0.0,0.0,17
100023,1499902,1499902,8,1499902.0,0.0,11999216,100023,100023,8,100023.0,...,8.505000e+11,0.0,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,8


In [17]:
credit = credit.replace([np.inf, -np.inf], np.nan)
###credit = credit.fillna(credit.max() + (credit.max() * 0.4))
credit.head()

,CARD_SK_ID_PREV_MIN,CARD_SK_ID_PREV_MAX,CARD_SK_ID_PREV_SIZE,CARD_SK_ID_PREV_MEAN,CARD_SK_ID_PREV_VAR,CARD_SK_ID_PREV_SUM,CARD_SK_ID_CURR_MIN,CARD_SK_ID_CURR_MAX,CARD_SK_ID_CURR_SIZE,CARD_SK_ID_CURR_MEAN,...,CARD_card_AMT_DRAWINGS_CURRENT__AMT_DRAWINGS_POS_CURRENT_VAR,CARD_card_AMT_DRAWINGS_CURRENT__AMT_DRAWINGS_POS_CURRENT_SUM,CARD_NAME_CONTRACT_STATUS_Active_MEAN,CARD_NAME_CONTRACT_STATUS_Completed_MEAN,CARD_NAME_CONTRACT_STATUS_Demand_MEAN,CARD_NAME_CONTRACT_STATUS_Signed_MEAN,CARD_NAME_CONTRACT_STATUS_Sent_proposal_MEAN,CARD_NAME_CONTRACT_STATUS_Refused_MEAN,CARD_NAME_CONTRACT_STATUS_Approved_MEAN,CARD_COUNT
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100006,1489396,1489396,6,1489396.0,0.0,8936376,100006,100006,6,100006.0,...,8.505000e+11,0.0,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,6
100011,1843384,1843384,74,1843384.0,0.0,136410416,100011,100011,74,100011.0,...,4.378378e+08,180000.0,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,74
100013,2038692,2038692,96,2038692.0,0.0,195714432,100013,100013,96,100013.0,...,8.249688e+08,571500.0,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,96
100021,2594025,2594025,17,2594025.0,0.0,44098425,100021,100021,17,100021.0,...,8.505000e+11,0.0,0.411865,0.588379,0.0,0.0,0.0,0.0,0.0,17
100023,1499902,1499902,8,1499902.0,0.0,11999216,100023,100023,8,100023.0,...,8.505000e+11,0.0,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,8


In [18]:
### application
df = FeatureEngineering.application_train_test(path, 'application_train.csv')
df.head()

Memory usage of dataframe: 404.30 MB
Memory usage after optimization: 169.87 MB
Decreased by 58.0%


,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,app_AMT_INCOME_TOTAL_div_CNT_CHILDREN,app_most_popular_AMT_GOODS_PRICE,app_popular_AMT_GOODS_PRICE,app_OWN_CAR_AGE_div_DAYS_BIRTH,app_OWN_CAR_AGE_div_DAYS_EMPLOYED,app_DAYS_LAST_PHONE_CHANGE_div_DAYS_BIRTH,app_DAYS_LAST_PHONE_CHANGE_div_DAYS_EMPLOYED,app_DAYS_EMPLOYED_PERC,app_DAYS_EMPLOYED_div_DAYS_BIRTH,app_CNT_CHILDREN_div_CNT_FAM_MEMBERS
0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,351000.0,...,202500.0,0,0,NaN,NaN,0.119873,1.780273,8824.0,0.067322,0.0
1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,1129500.0,...,270000.0,0,0,NaN,NaN,0.049377,0.696777,15576.0,0.070862,0.0
2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,135000.0,...,67500.0,0,0,-0.001365,-0.11554,0.042786,3.623047,18816.0,0.011810,0.0
3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,297000.0,...,135000.0,0,0,NaN,NaN,0.032471,0.203003,15968.0,0.159912,0.0
4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,513000.0,...,121500.0,0,0,NaN,NaN,0.055481,0.364014,16896.0,0.152466,0.0


In [19]:
df = df.replace([np.inf, -np.inf], np.nan)
#df = df.fillna(installments.max() + (installments.max() * 0.4))
df.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,app_AMT_INCOME_TOTAL_div_CNT_CHILDREN,app_most_popular_AMT_GOODS_PRICE,app_popular_AMT_GOODS_PRICE,app_OWN_CAR_AGE_div_DAYS_BIRTH,app_OWN_CAR_AGE_div_DAYS_EMPLOYED,app_DAYS_LAST_PHONE_CHANGE_div_DAYS_BIRTH,app_DAYS_LAST_PHONE_CHANGE_div_DAYS_EMPLOYED,app_DAYS_EMPLOYED_PERC,app_DAYS_EMPLOYED_div_DAYS_BIRTH,app_CNT_CHILDREN_div_CNT_FAM_MEMBERS
0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,351000.0,...,202500.0,0,0,NaN,NaN,0.119873,1.780273,8824.0,0.067322,0.0
1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,1129500.0,...,270000.0,0,0,NaN,NaN,0.049377,0.696777,15576.0,0.070862,0.0
2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,135000.0,...,67500.0,0,0,-0.001365,-0.11554,0.042786,3.623047,18816.0,0.011810,0.0
3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,297000.0,...,135000.0,0,0,NaN,NaN,0.032471,0.203003,15968.0,0.159912,0.0
4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,513000.0,...,121500.0,0,0,NaN,NaN,0.055481,0.364014,16896.0,0.152466,0.0


### データフレームを結合

In [20]:
### データフレームをマージ
df = pd.merge(df, bureau, on='SK_ID_CURR', how='left')
df = pd.merge(df, installments, on='SK_ID_CURR', how='left')
df = pd.merge(df, previous, on='SK_ID_CURR', how='left')
df = pd.merge(df, credit, on='SK_ID_CURR', how='left')
df = pd.merge(df, pos, on='SK_ID_CURR', how='left')
df.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,POS_NAME_CONTRACT_STATUS_Active_MEAN,POS_NAME_CONTRACT_STATUS_Completed_MEAN,POS_NAME_CONTRACT_STATUS_Signed_MEAN,POS_NAME_CONTRACT_STATUS_Approved_MEAN,POS_NAME_CONTRACT_STATUS_Returned_to_the_store_MEAN,POS_NAME_CONTRACT_STATUS_Demand_MEAN,POS_NAME_CONTRACT_STATUS_Canceled_MEAN,POS_NAME_CONTRACT_STATUS_XNA_MEAN,POS_NAME_CONTRACT_STATUS_Amortized_debt_MEAN,POS_COUNT
0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,351000.0,...,1.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,19.0
1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,1129500.0,...,0.928711,0.071411,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,28.0
2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,135000.0,...,0.750000,0.250000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,4.0
3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,297000.0,...,0.856934,0.095215,0.000000,0.0,0.047607,0.0,0.0,0.0,0.0,21.0
4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,513000.0,...,0.939453,0.045441,0.015152,0.0,0.000000,0.0,0.0,0.0,0.0,66.0


In [21]:
#df = FeatureEngineering.clean_data(df)
df.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,POS_NAME_CONTRACT_STATUS_Active_MEAN,POS_NAME_CONTRACT_STATUS_Completed_MEAN,POS_NAME_CONTRACT_STATUS_Signed_MEAN,POS_NAME_CONTRACT_STATUS_Approved_MEAN,POS_NAME_CONTRACT_STATUS_Returned_to_the_store_MEAN,POS_NAME_CONTRACT_STATUS_Demand_MEAN,POS_NAME_CONTRACT_STATUS_Canceled_MEAN,POS_NAME_CONTRACT_STATUS_XNA_MEAN,POS_NAME_CONTRACT_STATUS_Amortized_debt_MEAN,POS_COUNT
0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,351000.0,...,1.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,19.0
1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,1129500.0,...,0.928711,0.071411,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,28.0
2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,135000.0,...,0.750000,0.250000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,4.0
3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,297000.0,...,0.856934,0.095215,0.000000,0.0,0.047607,0.0,0.0,0.0,0.0,21.0
4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,513000.0,...,0.939453,0.045441,0.015152,0.0,0.000000,0.0,0.0,0.0,0.0,66.0


In [22]:
### 変数選択
df = FeatureEngineering2.data_post_processing(df)
print('='*50, '\n')
print('---=> df final shape:', df.shape, ' <=---', '\n')
print('=' * 50)
df.head()

---=> the DATA POST-PROCESSING is beginning, the dataset has 3396 features
names of feature are renamed
---=> pandas data types of features in the dataset are converted for a reduced memory usage
---=> 3279 features are remained after removing non-informative features
1142 features are eliminated by LightGBM classifier with an 0.7 auc score limit in step I
---=> 2137 features are remained after removing features not interesting for LightGBM classifier
---=> 90 features are generated with the risk_groupanizer
---=> the DATA POST-PROCESSING is ended!, now the dataset has a total 2227 features

---=> df final shape: (356249, 2227)  <=--- 



,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,CARD_MONTHS_BALANCE_MAX_high_risk,CARD_MONTHS_BALANCE_MAX_low_risk,CARD_AMT_DRAWINGS_OTHER_CURRENT_MIN_high_risk,CARD_AMT_DRAWINGS_OTHER_CURRENT_MIN_low_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CARD_card_missing_MIN_high_risk,CARD_card_missing_MIN_low_risk,CARD_card_missing_MAX_high_risk,CARD_card_missing_MAX_low_risk
0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,351000.0,...,0,0,0,0,0,0,0,0,0,0
1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,1129500.0,...,0,0,0,0,0,0,0,0,0,0
2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,135000.0,...,0,0,0,0,0,0,0,0,0,0
3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,297000.0,...,1,0,0,1,0,1,0,1,0,1
4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,513000.0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
# infをnanへ置換
df = df.replace([np.inf, -np.inf], np.nan)
df.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,CARD_MONTHS_BALANCE_MAX_high_risk,CARD_MONTHS_BALANCE_MAX_low_risk,CARD_AMT_DRAWINGS_OTHER_CURRENT_MIN_high_risk,CARD_AMT_DRAWINGS_OTHER_CURRENT_MIN_low_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CARD_card_missing_MIN_high_risk,CARD_card_missing_MIN_low_risk,CARD_card_missing_MAX_high_risk,CARD_card_missing_MAX_low_risk
0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,351000.0,...,0,0,0,0,0,0,0,0,0,0
1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,1129500.0,...,0,0,0,0,0,0,0,0,0,0
2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,135000.0,...,0,0,0,0,0,0,0,0,0,0
3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,297000.0,...,1,0,0,1,0,1,0,1,0,1
4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,513000.0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
### 変数名から不正な文字を除去
df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
print(df.shape)
df.head()

(356249, 2227)


,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,CARD_MONTHS_BALANCE_MAX_high_risk,CARD_MONTHS_BALANCE_MAX_low_risk,CARD_AMT_DRAWINGS_OTHER_CURRENT_MIN_high_risk,CARD_AMT_DRAWINGS_OTHER_CURRENT_MIN_low_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CARD_card_missing_MIN_high_risk,CARD_card_missing_MIN_low_risk,CARD_card_missing_MAX_high_risk,CARD_card_missing_MAX_low_risk
0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,351000.0,...,0,0,0,0,0,0,0,0,0,0
1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,1129500.0,...,0,0,0,0,0,0,0,0,0,0
2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,135000.0,...,0,0,0,0,0,0,0,0,0,0
3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,297000.0,...,1,0,0,1,0,1,0,1,0,1
4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,513000.0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
### split train, and test datasets
app_train = df[df['TARGET'].notnull()]
app_test = df[df['TARGET'].isnull()]

print(app_test.shape)
app_test.head()

(48744, 2227)


,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,CARD_MONTHS_BALANCE_MAX_high_risk,CARD_MONTHS_BALANCE_MAX_low_risk,CARD_AMT_DRAWINGS_OTHER_CURRENT_MIN_high_risk,CARD_AMT_DRAWINGS_OTHER_CURRENT_MIN_low_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CARD_card_missing_MIN_high_risk,CARD_card_missing_MIN_low_risk,CARD_card_missing_MAX_high_risk,CARD_card_missing_MAX_low_risk
307505,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,450000.0,...,0,0,0,0,0,0,0,0,0,0
307506,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,180000.0,...,0,0,0,0,0,0,0,0,0,0
307507,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,630000.0,...,1,0,1,0,1,0,1,0,0,1
307508,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,1575000.0,...,1,0,1,0,1,0,1,0,0,1
307509,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,625500.0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
### indexを削除
app_train=app_train.reset_index()
app_test=app_test.reset_index()
app_test.head()

,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CARD_MONTHS_BALANCE_MAX_high_risk,CARD_MONTHS_BALANCE_MAX_low_risk,CARD_AMT_DRAWINGS_OTHER_CURRENT_MIN_high_risk,CARD_AMT_DRAWINGS_OTHER_CURRENT_MIN_low_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CARD_card_missing_MIN_high_risk,CARD_card_missing_MIN_low_risk,CARD_card_missing_MAX_high_risk,CARD_card_missing_MAX_low_risk
0,307505,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0,0,0,0,0,0,0,0,0,0
1,307506,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0,0,0,0,0,0,0,0,0,0
2,307507,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,1,0,1,0,1,0,1,0,0,1
3,307508,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,1,0,1,0,1,0,1,0,0,1
4,307509,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
### indexを削除
app_train = app_train.drop(columns=['index'])
app_test = app_test.drop(columns=['index'])
app_test.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,CARD_MONTHS_BALANCE_MAX_high_risk,CARD_MONTHS_BALANCE_MAX_low_risk,CARD_AMT_DRAWINGS_OTHER_CURRENT_MIN_high_risk,CARD_AMT_DRAWINGS_OTHER_CURRENT_MIN_low_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CARD_card_missing_MIN_high_risk,CARD_card_missing_MIN_low_risk,CARD_card_missing_MAX_high_risk,CARD_card_missing_MAX_low_risk
0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,450000.0,...,0,0,0,0,0,0,0,0,0,0
1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,180000.0,...,0,0,0,0,0,0,0,0,0,0
2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,630000.0,...,1,0,1,0,1,0,1,0,0,1
3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,1575000.0,...,1,0,1,0,1,0,1,0,0,1
4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,625500.0,...,0,0,0,0,0,0,0,0,0,0


### modelを生成

In [28]:
# trainデータセット作成
x_train = app_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = app_train['TARGET']
id_train = app_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=='O':
        x_train[col] = x_train[col].astype('category')

In [29]:
# testデータセット作成
x_test = app_test.drop(columns=['SK_ID_CURR', 'TARGET'])
id_test = app_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [30]:
# infをnanへ置換
#app_train = app_train.fillna(app_train.max() + (app_train.max() * 0.4))
app_train = app_train.replace([np.inf, -np.inf], np.nan)
app_train.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,CARD_MONTHS_BALANCE_MAX_high_risk,CARD_MONTHS_BALANCE_MAX_low_risk,CARD_AMT_DRAWINGS_OTHER_CURRENT_MIN_high_risk,CARD_AMT_DRAWINGS_OTHER_CURRENT_MIN_low_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CARD_card_missing_MIN_high_risk,CARD_card_missing_MIN_low_risk,CARD_card_missing_MAX_high_risk,CARD_card_missing_MAX_low_risk
0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,351000.0,...,0,0,0,0,0,0,0,0,0,0
1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,1129500.0,...,0,0,0,0,0,0,0,0,0,0
2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,135000.0,...,0,0,0,0,0,0,0,0,0,0
3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,297000.0,...,1,0,0,1,0,1,0,1,0,1
4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,513000.0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
# 欠損値補完
app_test = app_test.replace([np.inf, -np.inf], np.nan)
#app_test = app_test.fillna(app_test.max() + (app_test.max() * 0.4))
app_test.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,CARD_MONTHS_BALANCE_MAX_high_risk,CARD_MONTHS_BALANCE_MAX_low_risk,CARD_AMT_DRAWINGS_OTHER_CURRENT_MIN_high_risk,CARD_AMT_DRAWINGS_OTHER_CURRENT_MIN_low_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CARD_card_missing_MIN_high_risk,CARD_card_missing_MIN_low_risk,CARD_card_missing_MAX_high_risk,CARD_card_missing_MAX_low_risk
0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,450000.0,...,0,0,0,0,0,0,0,0,0,0
1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,180000.0,...,0,0,0,0,0,0,0,0,0,0
2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,630000.0,...,1,0,1,0,1,0,1,0,0,1
3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,1575000.0,...,1,0,1,0,1,0,1,0,0,1
4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,625500.0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
### 処理対象の列名を配列化
encoding_columns = list(app_test.select_dtypes(include='number').columns)
encoding_columns.remove('SK_ID_CURR')
encoding_columns.remove('TARGET')

In [33]:
### indexを設定
app_train = app_train.set_index('SK_ID_CURR')
app_test = app_test.set_index('SK_ID_CURR')
app_test.head()

,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,...,CARD_MONTHS_BALANCE_MAX_high_risk,CARD_MONTHS_BALANCE_MAX_low_risk,CARD_AMT_DRAWINGS_OTHER_CURRENT_MIN_high_risk,CARD_AMT_DRAWINGS_OTHER_CURRENT_MIN_low_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CARD_card_missing_MIN_high_risk,CARD_card_missing_MIN_low_risk,CARD_card_missing_MAX_high_risk,CARD_card_missing_MAX_low_risk
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,450000.0,0.018845,...,0,0,0,0,0,0,0,0,0,0
100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,180000.0,0.035797,...,0,0,0,0,0,0,0,0,0,0
100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,630000.0,0.019104,...,1,0,1,0,1,0,1,0,0,1
100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,...,1,0,1,0,1,0,1,0,0,1
100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,625500.0,0.010033,...,0,0,0,0,0,0,0,0,0,0


In [34]:
### 標準化
for column in encoding_columns:
    num_cols = []
    num_cols.append(column)
    result = modules.process_standardization(app_train, app_test, num_cols)
        
app_test.head()

,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,...,CARD_MONTHS_BALANCE_MAX_high_risk,CARD_MONTHS_BALANCE_MAX_low_risk,CARD_AMT_DRAWINGS_OTHER_CURRENT_MIN_high_risk,CARD_AMT_DRAWINGS_OTHER_CURRENT_MIN_low_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CARD_card_missing_MIN_high_risk,CARD_card_missing_MIN_low_risk,CARD_card_missing_MAX_high_risk,CARD_card_missing_MAX_low_risk
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,NaN,0.720375,-0.717914,-0.664541,-0.577536,-0.307107,-0.075103,-0.451788,-0.239271,-0.146484,...,-0.593454,-0.150514,-0.498140,-0.302406,-0.498058,-0.302406,-0.498165,-0.302336,-0.323903,-0.48064
100005,NaN,-1.388165,-0.717914,-0.664541,-0.577536,-0.637955,-0.934824,-0.671917,-0.970093,1.079102,...,-0.593454,-0.150514,-0.498140,-0.302406,-0.498058,-0.302406,-0.498165,-0.302336,-0.323903,-0.48064
100013,NaN,-1.388165,1.392925,-0.664541,-0.577536,0.313233,0.159594,2.943901,0.247945,-0.127563,...,1.685051,-0.150514,2.007469,-0.302406,2.007797,-0.302406,2.007367,-0.302336,-0.323903,2.08056
100028,NaN,0.720375,-0.717914,-0.664541,2.192089,1.347132,2.424815,1.511670,2.805824,0.399658,...,1.685051,-0.150514,2.007469,-0.302406,2.007797,-0.302406,2.007367,-0.302336,-0.323903,2.08056
100038,NaN,-1.388165,1.392925,1.504799,0.807276,0.106453,0.065769,0.342102,0.235764,-0.783203,...,-0.593454,-0.150514,-0.498140,-0.302406,-0.498058,-0.302406,-0.498165,-0.302336,-0.323903,-0.48064


In [35]:
### indexを削除
app_train = app_train.reset_index()
app_test = app_test.reset_index()
app_test.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,CARD_MONTHS_BALANCE_MAX_high_risk,CARD_MONTHS_BALANCE_MAX_low_risk,CARD_AMT_DRAWINGS_OTHER_CURRENT_MIN_high_risk,CARD_AMT_DRAWINGS_OTHER_CURRENT_MIN_low_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CARD_card_missing_MIN_high_risk,CARD_card_missing_MIN_low_risk,CARD_card_missing_MAX_high_risk,CARD_card_missing_MAX_low_risk
0,100001,NaN,0.720375,-0.717914,-0.664541,-0.577536,-0.307107,-0.075103,-0.451788,-0.239271,...,-0.593454,-0.150514,-0.498140,-0.302406,-0.498058,-0.302406,-0.498165,-0.302336,-0.323903,-0.48064
1,100005,NaN,-1.388165,-0.717914,-0.664541,-0.577536,-0.637955,-0.934824,-0.671917,-0.970093,...,-0.593454,-0.150514,-0.498140,-0.302406,-0.498058,-0.302406,-0.498165,-0.302336,-0.323903,-0.48064
2,100013,NaN,-1.388165,1.392925,-0.664541,-0.577536,0.313233,0.159594,2.943901,0.247945,...,1.685051,-0.150514,2.007469,-0.302406,2.007797,-0.302406,2.007367,-0.302336,-0.323903,2.08056
3,100028,NaN,0.720375,-0.717914,-0.664541,2.192089,1.347132,2.424815,1.511670,2.805824,...,1.685051,-0.150514,2.007469,-0.302406,2.007797,-0.302406,2.007367,-0.302336,-0.323903,2.08056
4,100038,NaN,-1.388165,1.392925,1.504799,0.807276,0.106453,0.065769,0.342102,0.235764,...,-0.593454,-0.150514,-0.498140,-0.302406,-0.498058,-0.302406,-0.498165,-0.302336,-0.323903,-0.48064


In [36]:
app_train = FeatureEngineering2.reduce_mem_usage(app_train)
app_train.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,CARD_MONTHS_BALANCE_MAX_high_risk,CARD_MONTHS_BALANCE_MAX_low_risk,CARD_AMT_DRAWINGS_OTHER_CURRENT_MIN_high_risk,CARD_AMT_DRAWINGS_OTHER_CURRENT_MIN_low_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CARD_card_missing_MIN_high_risk,CARD_card_missing_MIN_low_risk,CARD_card_missing_MAX_high_risk,CARD_card_missing_MAX_low_risk
0,100002,1.0,-1.387695,-0.717773,-0.664551,-0.577637,0.313232,-0.478027,-0.166138,-0.507324,...,-0.593262,-0.150513,-0.498047,-0.302490,-0.498047,-0.302490,-0.498047,-0.302246,-0.323975,-0.480713
1,100003,0.0,0.720215,-0.717773,1.504883,-0.577637,0.933594,1.725586,0.592773,1.599609,...,-0.593262,-0.150513,-0.498047,-0.302490,-0.498047,-0.302490,-0.498047,-0.302246,-0.323975,-0.480713
2,100004,0.0,-1.387695,1.392578,-0.664551,-0.577637,-0.927246,-1.153320,-1.404297,-1.091797,...,-0.593262,-0.150513,-0.498047,-0.302490,-0.498047,-0.302490,-0.498047,-0.302246,-0.323975,-0.480713
3,100006,0.0,0.720215,-0.717773,-0.664551,-0.577637,-0.307129,-0.711426,0.177856,-0.653320,...,1.684570,-0.150513,-0.498047,3.306641,-0.498047,3.306641,-0.498047,3.306641,-0.323975,2.080078
4,100007,0.0,-1.387695,-0.717773,-0.664551,-0.577637,-0.431152,-0.213745,-0.361816,-0.068726,...,-0.593262,-0.150513,-0.498047,-0.302490,-0.498047,-0.302490,-0.498047,-0.302246,-0.323975,-0.480713


In [37]:
app_test = FeatureEngineering2.reduce_mem_usage(app_test)
app_test.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,CARD_MONTHS_BALANCE_MAX_high_risk,CARD_MONTHS_BALANCE_MAX_low_risk,CARD_AMT_DRAWINGS_OTHER_CURRENT_MIN_high_risk,CARD_AMT_DRAWINGS_OTHER_CURRENT_MIN_low_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CARD_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CARD_card_missing_MIN_high_risk,CARD_card_missing_MIN_low_risk,CARD_card_missing_MAX_high_risk,CARD_card_missing_MAX_low_risk
0,100001,NaN,0.720215,-0.717773,-0.664551,-0.577637,-0.307129,-0.075073,-0.451904,-0.239258,...,-0.593262,-0.150513,-0.498047,-0.30249,-0.498047,-0.30249,-0.498047,-0.302246,-0.323975,-0.480713
1,100005,NaN,-1.387695,-0.717773,-0.664551,-0.577637,-0.638184,-0.935059,-0.671875,-0.970215,...,-0.593262,-0.150513,-0.498047,-0.30249,-0.498047,-0.30249,-0.498047,-0.302246,-0.323975,-0.480713
2,100013,NaN,-1.387695,1.392578,-0.664551,-0.577637,0.313232,0.159546,2.943359,0.247925,...,1.684570,-0.150513,2.007812,-0.30249,2.007812,-0.30249,2.007812,-0.302246,-0.323975,2.080078
3,100028,NaN,0.720215,-0.717773,-0.664551,2.191406,1.346680,2.425781,1.511719,2.806641,...,1.684570,-0.150513,2.007812,-0.30249,2.007812,-0.30249,2.007812,-0.302246,-0.323975,2.080078
4,100038,NaN,-1.387695,1.392578,1.504883,0.807129,0.106445,0.065796,0.342041,0.235718,...,-0.593262,-0.150513,-0.498047,-0.30249,-0.498047,-0.30249,-0.498047,-0.302246,-0.323975,-0.480713


In [38]:
# trainデータセット作成
x_train = app_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = app_train['TARGET']
id_train = app_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=='O':
        x_train[col] = x_train[col].astype('category')

In [39]:
# testデータセット作成
x_test = app_test.drop(columns=['SK_ID_CURR', 'TARGET'])
id_test = app_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [40]:
### training Xgb
# 1層目のモデル
# pred_train_1a, pred_train_1bは、学習データのクロスバリデーションでの予測値
# pred_test_1a, pred_test_1bは、テストデータの予測値
model_1a = Model1Lgb()
pred_train_1a, pred_test_1a = predict_cv(model_1a, x_train, y_train, x_test)

[LightGBM] [Warning] Unknown parameter: silent
[LightGBM] [Warning] Unknown parameter: importance_type
[1]	train's auc: 0.736843	valid's auc: 0.728342
[2]	train's auc: 0.748613	valid's auc: 0.738768
[3]	train's auc: 0.750427	valid's auc: 0.741098
[4]	train's auc: 0.755296	valid's auc: 0.746332
[5]	train's auc: 0.755817	valid's auc: 0.746516
[6]	train's auc: 0.756773	valid's auc: 0.747611
[7]	train's auc: 0.757519	valid's auc: 0.748012
[8]	train's auc: 0.75794	valid's auc: 0.748179
[9]	train's auc: 0.758895	valid's auc: 0.749402
[10]	train's auc: 0.759447	valid's auc: 0.74977
[11]	train's auc: 0.759388	valid's auc: 0.749522
[12]	train's auc: 0.759572	valid's auc: 0.749598
[13]	train's auc: 0.759628	valid's auc: 0.749727
[14]	train's auc: 0.759664	valid's auc: 0.749489
[15]	train's auc: 0.759917	valid's auc: 0.749656
[16]	train's auc: 0.759795	valid's auc: 0.749433
[17]	train's auc: 0.76001	valid's auc: 0.749787
[18]	train's auc: 0.760201	valid's auc: 0.7498
[19]	train's auc: 0.760505	va

[166]	train's auc: 0.788569	valid's auc: 0.767408
[167]	train's auc: 0.788714	valid's auc: 0.767495
[168]	train's auc: 0.78889	valid's auc: 0.767578
[169]	train's auc: 0.789042	valid's auc: 0.767689
[170]	train's auc: 0.789183	valid's auc: 0.767797
[171]	train's auc: 0.789308	valid's auc: 0.767852
[172]	train's auc: 0.789479	valid's auc: 0.76794
[173]	train's auc: 0.789619	valid's auc: 0.768022
[174]	train's auc: 0.789785	valid's auc: 0.768143
[175]	train's auc: 0.789963	valid's auc: 0.768261
[176]	train's auc: 0.790133	valid's auc: 0.768374
[177]	train's auc: 0.790278	valid's auc: 0.768472
[178]	train's auc: 0.790418	valid's auc: 0.76855
[179]	train's auc: 0.790582	valid's auc: 0.768649
[180]	train's auc: 0.790747	valid's auc: 0.768744
[181]	train's auc: 0.790933	valid's auc: 0.768868
[182]	train's auc: 0.791085	valid's auc: 0.768962
[183]	train's auc: 0.791226	valid's auc: 0.769051
[184]	train's auc: 0.791366	valid's auc: 0.769109
[185]	train's auc: 0.791558	valid's auc: 0.769211
[18

[331]	train's auc: 0.811409	valid's auc: 0.779828
[332]	train's auc: 0.81151	valid's auc: 0.779873
[333]	train's auc: 0.811641	valid's auc: 0.779957
[334]	train's auc: 0.811765	valid's auc: 0.780026
[335]	train's auc: 0.811906	valid's auc: 0.780087
[336]	train's auc: 0.812032	valid's auc: 0.780142
[337]	train's auc: 0.812146	valid's auc: 0.780203
[338]	train's auc: 0.812269	valid's auc: 0.780255
[339]	train's auc: 0.812388	valid's auc: 0.780313
[340]	train's auc: 0.812501	valid's auc: 0.780364
[341]	train's auc: 0.812629	valid's auc: 0.780439
[342]	train's auc: 0.812759	valid's auc: 0.78049
[343]	train's auc: 0.812877	valid's auc: 0.780556
[344]	train's auc: 0.812989	valid's auc: 0.7806
[345]	train's auc: 0.813108	valid's auc: 0.78066
[346]	train's auc: 0.813212	valid's auc: 0.780704
[347]	train's auc: 0.813334	valid's auc: 0.780755
[348]	train's auc: 0.813451	valid's auc: 0.780832
[349]	train's auc: 0.81357	valid's auc: 0.780865
[350]	train's auc: 0.813697	valid's auc: 0.780935
[351]	

[496]	train's auc: 0.828302	valid's auc: 0.786295
[497]	train's auc: 0.828403	valid's auc: 0.786324
[498]	train's auc: 0.828485	valid's auc: 0.786353
[499]	train's auc: 0.828579	valid's auc: 0.786375
[500]	train's auc: 0.828662	valid's auc: 0.786382
[501]	train's auc: 0.828759	valid's auc: 0.786401
[502]	train's auc: 0.828857	valid's auc: 0.786433
[503]	train's auc: 0.828972	valid's auc: 0.786474
[504]	train's auc: 0.829061	valid's auc: 0.786504
[505]	train's auc: 0.829152	valid's auc: 0.786531
[506]	train's auc: 0.829241	valid's auc: 0.78655
[507]	train's auc: 0.829319	valid's auc: 0.786565
[508]	train's auc: 0.829425	valid's auc: 0.786591
[509]	train's auc: 0.829512	valid's auc: 0.786614
[510]	train's auc: 0.829601	valid's auc: 0.786636
[511]	train's auc: 0.829689	valid's auc: 0.78666
[512]	train's auc: 0.829781	valid's auc: 0.786681
[513]	train's auc: 0.82987	valid's auc: 0.786714
[514]	train's auc: 0.829953	valid's auc: 0.786726
[515]	train's auc: 0.830034	valid's auc: 0.786759
[51

[661]	train's auc: 0.842394	valid's auc: 0.789865
[662]	train's auc: 0.842469	valid's auc: 0.789878
[663]	train's auc: 0.842548	valid's auc: 0.789896
[664]	train's auc: 0.842616	valid's auc: 0.789915
[665]	train's auc: 0.842685	valid's auc: 0.789931
[666]	train's auc: 0.842754	valid's auc: 0.789937
[667]	train's auc: 0.842832	valid's auc: 0.789955
[668]	train's auc: 0.84291	valid's auc: 0.789961
[669]	train's auc: 0.843002	valid's auc: 0.789984
[670]	train's auc: 0.843073	valid's auc: 0.789995
[671]	train's auc: 0.843159	valid's auc: 0.790017
[672]	train's auc: 0.843229	valid's auc: 0.790034
[673]	train's auc: 0.843314	valid's auc: 0.790047
[674]	train's auc: 0.843383	valid's auc: 0.790049
[675]	train's auc: 0.84345	valid's auc: 0.790046
[676]	train's auc: 0.843542	valid's auc: 0.790062
[677]	train's auc: 0.843619	valid's auc: 0.790084
[678]	train's auc: 0.843711	valid's auc: 0.790096
[679]	train's auc: 0.843803	valid's auc: 0.790125
[680]	train's auc: 0.843869	valid's auc: 0.790141
[6

...

[4350]	train's auc: 0.970557	valid's auc: 0.793451
[4351]	train's auc: 0.970567	valid's auc: 0.793453
[4352]	train's auc: 0.970581	valid's auc: 0.793454
[4353]	train's auc: 0.970596	valid's auc: 0.793449
[4354]	train's auc: 0.970616	valid's auc: 0.793447
[4355]	train's auc: 0.97063	valid's auc: 0.793446
[4356]	train's auc: 0.970646	valid's auc: 0.793446
[4357]	train's auc: 0.970663	valid's auc: 0.793452
[4358]	train's auc: 0.970678	valid's auc: 0.793455
[4359]	train's auc: 0.97069	valid's auc: 0.793458
[4360]	train's auc: 0.970703	valid's auc: 0.793463
[4361]	train's auc: 0.970716	valid's auc: 0.793462
[4362]	train's auc: 0.970731	valid's auc: 0.793464
[4363]	train's auc: 0.97074	valid's auc: 0.793458
[4364]	train's auc: 0.970755	valid's auc: 0.793464
[4365]	train's auc: 0.970769	valid's auc: 0.793472
[4366]	train's auc: 0.970785	valid's auc: 0.793469
[4367]	train's auc: 0.970799	valid's auc: 0.793479
[4368]	train's auc: 0.970808	valid's auc: 0.793479
[4369]	train's auc: 0.970823	valid

[4512]	train's auc: 0.972709	valid's auc: 0.793298
[4513]	train's auc: 0.972722	valid's auc: 0.793291
[4514]	train's auc: 0.97274	valid's auc: 0.793292
[4515]	train's auc: 0.972759	valid's auc: 0.79329
[4516]	train's auc: 0.972775	valid's auc: 0.79329
[4517]	train's auc: 0.972784	valid's auc: 0.793293
[4518]	train's auc: 0.972801	valid's auc: 0.79329
[4519]	train's auc: 0.972816	valid's auc: 0.793296
[4520]	train's auc: 0.972825	valid's auc: 0.793293
[4521]	train's auc: 0.972835	valid's auc: 0.793286
[4522]	train's auc: 0.972847	valid's auc: 0.793284
[4523]	train's auc: 0.972862	valid's auc: 0.793278
[4524]	train's auc: 0.972873	valid's auc: 0.793272
[4525]	train's auc: 0.972884	valid's auc: 0.793266
[4526]	train's auc: 0.972895	valid's auc: 0.793264
[4527]	train's auc: 0.972908	valid's auc: 0.793263
[4528]	train's auc: 0.972918	valid's auc: 0.793264
[4529]	train's auc: 0.972931	valid's auc: 0.793267
[4530]	train's auc: 0.97294	valid's auc: 0.793269
[4531]	train's auc: 0.972956	valid's

[4674]	train's auc: 0.974763	valid's auc: 0.793111
[4675]	train's auc: 0.974771	valid's auc: 0.793111
[4676]	train's auc: 0.974781	valid's auc: 0.793112
[4677]	train's auc: 0.97479	valid's auc: 0.793118
[4678]	train's auc: 0.9748	valid's auc: 0.793122
[4679]	train's auc: 0.974813	valid's auc: 0.793121
[4680]	train's auc: 0.974825	valid's auc: 0.793117
[4681]	train's auc: 0.974836	valid's auc: 0.793122
[4682]	train's auc: 0.974847	valid's auc: 0.793122
[4683]	train's auc: 0.974859	valid's auc: 0.793127
[4684]	train's auc: 0.974867	valid's auc: 0.793123
[4685]	train's auc: 0.974878	valid's auc: 0.793124
[4686]	train's auc: 0.974888	valid's auc: 0.793118
[4687]	train's auc: 0.974905	valid's auc: 0.793116
[4688]	train's auc: 0.974918	valid's auc: 0.793114
[4689]	train's auc: 0.974928	valid's auc: 0.793113
[4690]	train's auc: 0.974938	valid's auc: 0.793106
[4691]	train's auc: 0.974947	valid's auc: 0.793102
[4692]	train's auc: 0.97496	valid's auc: 0.793105
[4693]	train's auc: 0.974974	valid'

[4836]	train's auc: 0.976578	valid's auc: 0.793013
[4837]	train's auc: 0.976592	valid's auc: 0.793014
[4838]	train's auc: 0.976604	valid's auc: 0.793011
[4839]	train's auc: 0.976613	valid's auc: 0.793011
[4840]	train's auc: 0.976624	valid's auc: 0.793007
[4841]	train's auc: 0.976634	valid's auc: 0.793015
[4842]	train's auc: 0.976643	valid's auc: 0.793006
[4843]	train's auc: 0.976651	valid's auc: 0.793001
[4844]	train's auc: 0.976661	valid's auc: 0.792995
[4845]	train's auc: 0.976673	valid's auc: 0.792992
[4846]	train's auc: 0.976681	valid's auc: 0.792989
[4847]	train's auc: 0.976688	valid's auc: 0.792984
[4848]	train's auc: 0.976699	valid's auc: 0.792983
[4849]	train's auc: 0.976709	valid's auc: 0.792985
[4850]	train's auc: 0.976722	valid's auc: 0.792979
[4851]	train's auc: 0.976731	valid's auc: 0.792981
[4852]	train's auc: 0.97674	valid's auc: 0.792987
[4853]	train's auc: 0.976756	valid's auc: 0.792984
[4854]	train's auc: 0.976764	valid's auc: 0.792985
[4855]	train's auc: 0.976774	val

[4998]	train's auc: 0.978348	valid's auc: 0.792923
[4999]	train's auc: 0.978362	valid's auc: 0.792919
[5000]	train's auc: 0.978372	valid's auc: 0.792911
[0.17702347 0.03068465 0.04180998 ... 0.02008379 0.02589456 0.0319165 ]


In [41]:
pred_train_1a

array([0.17702347, 0.01235432, 0.04068393, ..., 0.0319165 , 0.04749503,
       0.05135314])

In [42]:
pred_test_1a

array([0.02919509, 0.10104046, 0.03154963, ..., 0.00833822, 0.01940726,
       0.22990706])

In [43]:
pred_1a = []
for a in np.array(pred_train_1a):
    pred_1a.append(a)

len(pred_1a)

307505

In [44]:
print(type(pred_1a))

<class 'list'>


In [45]:
### 予測値を特徴量としてデータフレームを作成
sk_id_curr = app_train['SK_ID_CURR'].copy().to_list()
train_x_2 = pd.DataFrame({'SK_ID_CURR': sk_id_curr, 'set1': pred_1a})
print(train_x_2.shape)
train_x_2.head()

(307505, 2)


,SK_ID_CURR,pred_1a
0,100002,0.177023
1,100003,0.012354
2,100004,0.040684
3,100006,0.030685
4,100007,0.041810


In [46]:
train_x_2.to_csv(
    path_or_buf="./home-credit-default-risk/exports/pred_train_set1a.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

In [47]:
pred_1a = []
for a in np.array(pred_test_1a):
    pred_1a.append(a)

#pred_1b = np.array(pred_1b)
len(pred_1a)

48744

In [48]:
### 予測値を特徴量としてデータフレームを作成
sk_id_curr = app_test['SK_ID_CURR'].copy().to_list()
test_x_2 = pd.DataFrame({'SK_ID_CURR': sk_id_curr, 'set1': pred_1a})
print(test_x_2.shape)
test_x_2.head()

(48744, 2)


,SK_ID_CURR,pred_1b
0,100001,0.029195
1,100005,0.101040
2,100013,0.031550
3,100028,0.040153
4,100038,0.115963


In [49]:
test_x_2.to_csv(
    path_or_buf="./home-credit-default-risk/exports/pred_test_set1a.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

In [50]:
#del df
#del x_train
#del y_train
#del x_test
#del app_train
#del app_test
#del model_1b
#gc.collect()

## 特徴量生成2